# Qiskit Runtime REST API query & job submission

## Query API without authentication

In [1]:
import requests

url = 'https://runtime-us-east.quantum-computing.ibm.com/programs'
response = requests.get(url)

if response.status_code == 200:
    data = response.json()
else:
    print(f"Error: {response.status_code}")

[program['id'] for program in data['programs']]

['qasm3-runner', 'sampler', 'estimator', 'circuit-runner']

## Get temporary Access token from Auth API via API Token

In [2]:
import requests

with open('token') as file:
    token=file.read()

url = 'https://auth.quantum-computing.ibm.com/api/users/loginWithToken'
input={'apiToken': token}
auth_response = requests.post(url, json=input)
auth_id=auth_response.json()['id']

## GET available backends

In [3]:
url_backends = 'https://runtime-us-east.quantum-computing.ibm.com/backends'
headers = {'Content-Type': 'application/json',
            'x-access-token':auth_id}

backends_response = requests.get(url_backends, headers=headers)

print(backends_response.json()['devices'][:5],"...")

['ibmq_qasm_simulator', 'ibmq_qasm_simulator', 'simulator_extended_stabilizer', 'simulator_statevector', 'simulator_mps'] ...


## Run a job

### set up qasm circuit

In [4]:
qasm_string='''
OPENQASM 3;
include "stdgates.inc";
qreg q[1];
creg c[1];
x q[0];
c[0] = measure q[0]; 
'''   

### run circuit via API

In [5]:
import requests

url = 'https://runtime-us-east.quantum-computing.ibm.com/jobs'
headers = {'Content-Type': 'application/json','x-access-token':auth_id}
job_input = {
    'program_id': 'sampler',
    "backend": 'ibm_kyiv',
    "hub": "ibm-q-internal",
    "group": "dev-sys-software",
    "project": "internal-test",
    "start_session": "True", #set to False if you just need to run a single job
    "params": {
        "circuits": [qasm_string]*5
}}

response = requests.post(url, headers=headers, json=job_input)

if response.status_code == 200:
    job_id = response.json().get('id')
    print("Job created:",response.text)
else:
    print(f"Error: {response.status_code}")

Job created: {"id":"cn4dg6g3r3vg008f9mwg","backend":"ibm_kyiv","session_id":"cn4dg6g3r3vg008f9mwg"}


### Run follow-up jobs in the same Session

In [ ]:
session_id = job_id #any Qiskit Runtime Session carries the ID of the first job

In [7]:
headers = {'Content-Type': 'application/json','x-access-token':auth_id}
job_input = {
    'program_id': 'sampler',
    "backend": 'ibm_kyiv',
    "hub": "ibm-q-internal",
    "group": "dev-sys-software",
    "project": "internal-test",
    "session_id": session_id,
    "params": {
        "circuits": [qasm_string]*5
}}

response = requests.post(url, headers=headers, json=job_input)

if response.status_code == 200:
    job_id = response.json().get('id')
    print("Job created:",response.text)
else:
    print(f"Error: {response.status_code}")

Job created: {"id":"cn4dgs3vayrg008en3ag","backend":"ibm_kyiv","session_id":"cn4dg6g3r3vg008f9mwg"}


### Query Websocket for live job update

In [88]:
import asyncio
import websockets

with open('token') as file:
    api_key=file.read()

async def receive_status_updates(api_key):
    uri = "wss://runtime-us-east.quantum-computing.ibm.com/stream/jobs/"+job_id 
    extra_headers = {'Authorization': f'Bearer {api_key}'}

    async with websockets.connect(uri, extra_headers=extra_headers) as websocket:
        while True:
            try:
                message = await websocket.recv()
                print(f"Received message: {message}")  
            except websockets.exceptions.ConnectionClosed:
                print("WebSocket connection closed.")
                #await asyncio.sleep(5)  # Wait before attempting to reconnect
                break

await receive_status_updates(api_key)

Received message: Listening to results for job: ckr4vtnnetonc3t62v5g

Received message: {"quasi_dists": [{"1": 1.0}, {"1": 1.0}, {"1": 1.0}, {"1": 1.0}, {"1": 1.0}], "metadata": [{"shots": 1024, "readout_mitigation_overhead": 1.0, "readout_mitigation_time": 0.04957489899970824, "circuit_metadata": {}}, {"shots": 1024, "readout_mitigation_overhead": 1.0, "readout_mitigation_time": 0.021988247004628647, "circuit_metadata": {}}, {"shots": 1024, "readout_mitigation_overhead": 1.0, "readout_mitigation_time": 0.03040369199879933, "circuit_metadata": {}}, {"shots": 1024, "readout_mitigation_overhead": 1.0, "readout_mitigation_time": 0.03431434999947669, "circuit_metadata": {}}, {"shots": 1024, "readout_mitigation_overhead": 1.0, "readout_mitigation_time": 0.022905523001099937, "circuit_metadata": {}}]}
WebSocket connection closed.


## Check job status

In [89]:
status_result = requests.get(url+'/'+job_id,headers=headers)
print(status_result.json()['state'],job_id)

{'status': 'Completed'} ckr4vtnnetonc3t62v5g


### Get results

In [90]:
response_result= requests.get(url+'/'+job_id+'/results', headers=headers)

response_result.json()

{'quasi_dists': [{'1': 1.0}, {'1': 1.0}, {'1': 1.0}, {'1': 1.0}, {'1': 1.0}],
 'metadata': [{'shots': 1024,
   'readout_mitigation_overhead': 1.0,
   'readout_mitigation_time': 0.04957489899970824,
   'circuit_metadata': {}},
  {'shots': 1024,
   'readout_mitigation_overhead': 1.0,
   'readout_mitigation_time': 0.021988247004628647,
   'circuit_metadata': {}},
  {'shots': 1024,
   'readout_mitigation_overhead': 1.0,
   'readout_mitigation_time': 0.03040369199879933,
   'circuit_metadata': {}},
  {'shots': 1024,
   'readout_mitigation_overhead': 1.0,
   'readout_mitigation_time': 0.03431434999947669,
   'circuit_metadata': {}},
  {'shots': 1024,
   'readout_mitigation_overhead': 1.0,
   'readout_mitigation_time': 0.022905523001099937,
   'circuit_metadata': {}}]}

## Invalidate Token

In [91]:
logout_url = 'https://auth.quantum-computing.ibm.com/api/users/logout'
headers = {'x-access-token':auth_id}
logout_response = requests.post(logout_url, headers=headers)
print("response ok?:",logout_response.ok,logout_response.text)

response ok?: True 


## Test Token Invalidation

This should yield an Authoritzation error (Error 401) once access token is invalidated

In [92]:
rl = 'https://runtime-us-east.quantum-computing.ibm.com/jobs'
headers = {'Content-Type': 'application/json','x-access-token':auth_id}
job_input = {
    'program_id': 'sampler',
    "backend": 'ibmq_qasm_simulator',
    "hub": "ibm-q-internal",
    "group": "dev-sys-software",
    "project": "internal-test",
    "params": {
        "circuits": [qasm_string]
}}

response = requests.post(url, headers=headers, json=job_input)

if response.status_code == 200:
    job_id = response.json().get('id')
    print("Job created:",response.text)
else:
    print(response.text,"\n")
    print(f"Error: {response.status_code}")

{"errors":[{"code":1201,"message":"Invalid credentials.","solution":"Verify your credentials and try again.","more_info":"https://docs.quantum-computing.ibm.com/errors"}]} 

Error: 401
